# Wikipedia网页浏览数据的序列模式挖掘示例



数据来自https://snap.stanford.edu/data/wikispeedia.html 。

用户的Wikipedia的页面浏览记录，共51318个序列。
序列中的每个事件都是一个Wikipedia的页面浏览。

In [60]:
import csv
dataset = []

# 数据处理：整理序列数据为算法所需的list of lists的形式
for line in csv.reader((row for row in open("wikidata/paths_finished.tsv") if not row.startswith('#')), delimiter='\t'):
    if len(line) == 0:
        continue
    seq = line[3].split(";")
    seq = [x for x in seq if x != "<"]  # remove back clicks
    dataset.append(seq)
    
len(dataset) # 共51318个序列 
len(set(item for sequence in dataset for item in sequence)) # 序列数据中有4169种事件
dataset[0:3]

[['14th_century',
  '15th_century',
  '16th_century',
  'Pacific_Ocean',
  'Atlantic_Ocean',
  'Accra',
  'Africa',
  'Atlantic_slave_trade',
  'African_slave_trade'],
 ['14th_century',
  'Europe',
  'Africa',
  'Atlantic_slave_trade',
  'African_slave_trade'],
 ['14th_century',
  'Niger',
  'Nigeria',
  'British_Empire',
  'Slavery',
  'Africa',
  'Atlantic_slave_trade',
  'African_slave_trade']]

In [61]:
import pandas as pd
seqLen = [len(sequence) for sequence in dataset]
seqLend=pd.DataFrame(seqLen)
seqLend.describe()      ## 序列长度的平均数、中位数均为6个事件

,0
count,51318.000000
mean,6.356775
std,3.775778
min,1.000000
25%,4.000000
50%,6.000000
75%,7.000000
max,420.000000


## prefixspan包

使用由chuancong gao开发的prefixspan包，见https://pypi.org/project/prefixspan/ 。

该package对PrefixSpan算法的核心实现如下：

In [62]:
from collections import defaultdict

def frequent_rec(patt, mdb):  ##  patt是当前的频繁序列模式（前缀），而mdb是其投影序列的 (id, position) 元组的list。 (id, position) 为 （序列编号，从该序列的第几个位置开始）。
    results.append((len(mdb), patt))
    occurs = defaultdict(list)  ##  key为元素；value为该元素在每个序列中首次出现的位置，即多个(id, position) 的元组
    
    for (i, startpos) in mdb:       
        seq = db[i]      #  遍历投影数据中的所有序列
        for j in range(startpos + 1, len(seq)):       #  遍历投影序列的元素（从前缀出现的位置的后一位开始）
            l = occurs[seq[j]]
            if len(l) == 0 or l[-1][0] != i:  #  如果一个序列中包含该元素，记录该元素在该序列中首次出现的位置
                l.append((i, j))  
    
    for (c, newmdb) in occurs.items():
        if len(newmdb) >= minsup:     # newmdb里有多少个(id, position) 的元组，就代表有多少个序列包含该元素，所以len(newmdb) 为该元素在投影数据中的支持度。
            frequent_rec(patt + [c], newmdb) #对上次投影数据中的每个支持度大于等于最小支持度的元素，都再次调用函数构造其投影

db = [
    [0, 1, 2, 3, 4],
    [1, 1, 1, 3, 4],
    [2, 1, 2, 2, 0],
    [1, 1, 1, 2, 2],
]

minsup = 2
results = []
frequent_rec([], [(i, -1) for i in range(len(db))])
results

[(4, []),
 (2, [0]),
 (4, [1]),
 (3, [1, 2]),
 (2, [1, 2, 2]),
 (2, [1, 3]),
 (2, [1, 3, 4]),
 (2, [1, 4]),
 (2, [1, 1]),
 (2, [1, 1, 1]),
 (3, [2]),
 (2, [2, 2]),
 (2, [3]),
 (2, [3, 4]),
 (2, [4])]

In [63]:
"""
该prefixspan包是目前python中序列模式挖掘较为高效的实现。
"""
from prefixspan import PrefixSpan 

#### 1）通过 frequent(minsup) 输出满足最小支持度的序列模式。

In [64]:
#设定支持度为1000
%time PrefixSpan(dataset).frequent(1000) # 仅用时570ms，是现有的算法实现中效率最高的

Wall time: 501 ms


[(1297, ['Atlantic_Ocean']),
 (2738, ['Africa']),
 (4303, ['Europe']),
 (1861, ['North_America']),
 (8675, ['United_States']),
 (1110, ['China']),
 (1479, ['Science']),
 (1074, ['Christianity']),
 (1216, ['India']),
 (1588, ['France']),
 (2267, ['World_War_II']),
 (3261, ['England']),
 (1394, ['Periodic_table']),
 (1414, ['English_language']),
 (1050, ['United_Nations']),
 (1568, ['Mammal']),
 (1182, ['Bird']),
 (1738, ['Germany']),
 (3176, ['Earth']),
 (1666, ['Animal']),
 (1528, ['Computer']),
 (1023, ['Internet']),
 (3807, ['United_Kingdom']),
 (1007, ['Russia']),
 (1127, ['Plant']),
 (1167, ['Asia']),
 (1604, ['Human']),
 (1070, ['Japan']),
 (1316, ['Brain']),
 (1043, ['Brain', 'Telephone']),
 (1147, ['Agriculture']),
 (1034, ['Theatre']),
 (1041, ['Zebra']),
 (1171, ['Asteroid']),
 (1043, ['Asteroid', 'Viking']),
 (1192, ['Viking']),
 (1251, ['Telephone'])]

#### 2）通过 topk(n) 输出支持度排名前n的序列模式。

In [65]:
psResult = PrefixSpan(dataset)
psResult.topk(5) ## 输出支持度排名前5的序列

[(8675, ['United_States']),
 (4303, ['Europe']),
 (3807, ['United_Kingdom']),
 (3261, ['England']),
 (3176, ['Earth'])]

#### 3）通过Filter Function来添加约束，从而产生更有意义的序列模式。

*如：设置序列长度的下限；仅返回频繁的闭合序列 等。

##### 添加序列模式长度的限制

In [66]:
## 输出长度大于1的序列中，支持度排名前10的
psResult.topk(10, filter=lambda patt, matches: len(patt)>1) # patt is the current pattern and matches is the current list of matching sequence (id, position) tuples

[(1043, ['Asteroid', 'Viking']),
 (1043, ['Brain', 'Telephone']),
 (905, ['Theatre', 'Zebra']),
 (642, ['Pyramid', 'Bean']),
 (591, ['North_America', 'United_States']),
 (579, ['United_States', 'President_of_the_United_States']),
 (557, ['Animal', 'Mammal']),
 (543, ['Communication', 'Telephone']),
 (535, ['Europe', 'United_Kingdom']),
 (531, ['Mammal', 'Zebra'])]

In [67]:
## 输出长度大于3的序列中，支持度排名前10的
psResult.topk(10, filter=lambda patt, matches: len(patt)>3) 

[(285, ['Theatre', 'Africa', 'Lion', 'Zebra']),
 (271, ['Theatre', 'Animal', 'Mammal', 'Zebra']),
 (256, ['Brain', 'Computer_science', 'Communication', 'Telephone']),
 (246, ['Brain', 'Computer_science', 'Information', 'Telephone']),
 (239, ['Brain', 'Information', 'Communication', 'Telephone']),
 (226, ['Computer_science', 'Information', 'Communication', 'Telephone']),
 (225, ['Brain', 'Computer_science', 'Information', 'Communication']),
 (225,
  ['Brain', 'Computer_science', 'Information', 'Communication', 'Telephone']),
 (208, ['Asteroid', 'Earth', 'Europe', 'Viking']),
 (197, ['Asteroid', 'Europe', 'Norway', 'Viking'])]

*在设定minsup（最小支持度）之前，可以先用有filter的topk函数看一下长度不同的序列的支持度情况，然后再根据需要挖掘出的序列长度来设定最小支持度。

##### 增加闭合序列限制
*闭合序列：当一个序列的所有超集序列的支持度都小于该序列时，该序列为闭合序列。

In [68]:
## 输出长度大于3的序列中，支持度排名前10的闭合序列
psResult.topk(10, closed=True,filter=lambda patt, matches: len(patt)>3) 
"""
相较上面没有加closed=True的结果，增加了闭合序列限制的结果更有意义：
如，['Brain', 'Computer_science', 'Information', 'Communication'] 序列
和 ['Brain', 'Computer_science', 'Information', 'Communication', 'Telephone'] 序列具有相同的支持度225，
增加了closed=True的限制后，仅输出['Brain', 'Computer_science', 'Information', 'Communication', 'Telephone'] 
"""

[(285, ['Theatre', 'Africa', 'Lion', 'Zebra']),
 (271, ['Theatre', 'Animal', 'Mammal', 'Zebra']),
 (256, ['Brain', 'Computer_science', 'Communication', 'Telephone']),
 (246, ['Brain', 'Computer_science', 'Information', 'Telephone']),
 (239, ['Brain', 'Information', 'Communication', 'Telephone']),
 (226, ['Computer_science', 'Information', 'Communication', 'Telephone']),
 (225,
  ['Brain', 'Computer_science', 'Information', 'Communication', 'Telephone']),
 (208, ['Asteroid', 'Earth', 'Europe', 'Viking']),
 (197, ['Asteroid', 'Europe', 'Norway', 'Viking']),
 (182, ['Brain', 'Computer_science', 'Internet', 'Telephone'])]

In [76]:
"""
闭合频繁序列：当一个频繁序列的所有超集序列的支持度都小于该频繁序列时，该频繁序列为闭合频繁序列（frequent closed sequence）
"""

## 当最小支持度为200时，长度大于3的闭合频繁序列
sorted(psResult.frequent(200, closed=True, filter=lambda patt, matches: len(patt)>3) , reverse = True)# 按支持度降序排序输出

[(226, ['Computer_science', 'Information', 'Communication', 'Telephone']),
 (225,
  ['Brain', 'Computer_science', 'Information', 'Communication', 'Telephone']),
 (246, ['Brain', 'Computer_science', 'Information', 'Telephone']),
 (256, ['Brain', 'Computer_science', 'Communication', 'Telephone']),
 (239, ['Brain', 'Information', 'Communication', 'Telephone']),
 (271, ['Theatre', 'Animal', 'Mammal', 'Zebra']),
 (285, ['Theatre', 'Africa', 'Lion', 'Zebra']),
 (208, ['Asteroid', 'Earth', 'Europe', 'Viking'])]